In [ ]:
#————————————————————

# Name: Azure OpenAI Fine Tuning (V1)

# Purpose:


# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

# General Sources:

# Azure Authentication Token
# https://learn.microsoft.com/en-us/cli/azure/account?view=azure-cli-latest#az-account-get-access-token()

# Azure Openai Usage:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=python-new%2Ccommand-line
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/fine-tuning?tabs=turbo%2Cpython&pivots=programming-language-python
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/chatgpt?tabs=python&pivots=programming-language-chat-completions#few-shot-learning-with-chat-completion

# Additionals:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models
# https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8

# Download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [2]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI
import json
from IPython.display import clear_output
from IPython.core.display import HTML
import requests
import random

In [3]:
# Load required variables from env file.
load_dotenv(dotenv_path=Path("C:\Python\openai-lab\.venv\.env"))

# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [4]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    api_version = "2024-02-15-preview",
    azure_endpoint = azure_oai_endpoint
    )

In [40]:
recipe = 'Recipe: Asian-Inspired Tofu Avocado Curry\n\nIngredients:\n- 1 ripe avocado, pitted and sliced\n- 1 can of coconut milk\n- 200g tofu, cubed\n- 2 tablespoons soy sauce\n- 1 tablespoon olive oil\n- 1 onion, thinly sliced\n- 2 cloves garlic, minced\n- 1 teaspoon ginger, grated\n- 1 teaspoon curry powder\n- 1 teaspoon turmeric\n- 1 teaspoon paprika\n- Fresh cilantro, chopped for garnish\n- Steamed rice, for serving\n\nInstructions:\n1. In a large deep skillet or wok, heat olive oil over medium heat. Add onion, garlic, and ginger. Sauté until the onion becomes translucent.\n2. Add the tofu cubes to the skillet and cook until slightly browned on all sides.\n3. Add curry powder, turmeric, and paprika. Stir to evenly coat tofu and vegetables.\n4. Pour in the coconut milk and bring to a gentle simmer. Allow the flavors to meld for about 5 minutes.\n5. Season with soy sauce and stir well.\n6. Gently fold in avocado slices and cook for an additional 2 minutes.\n7. Serve over steamed rice and garnish with fresh cilantro.\n\nNote: This dish can be made spicier by adding a few slices of red chili or a sprinkle of chili flakes. A side of naan bread or roti can also be added for sopping up the rich sauce.'

In [41]:
entity_extraction = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[{"role":"system","content":'''You are an assistant designed to extract entities from text. Users will paste in a string of text and you will respond with entities you've extracted from the text as a JSON object. Here's an example of your output format:{"recipe_name": "", "ingredients": [{"name": "","amount": "" ,"unit": ""},{"name": "","amount": "" ,"unit": ""}]}'''},
                {"role":"user","content":"Ingredients:\n- 1 cucumber\n- 1 capsicum (any color)\n- 1 cup flour (or gluten-free flour)\n- 2 tablespoons soy sauce\n- Oil for frying\n\nInstructions:\n1. Prepare the vegetables:\n   - Slice the cucumber and capsicum into thin, even strips.\n   - Set them aside.\n\n2. Make the batter:\n   - In a mixing bowl, combine the flour and soy sauce.\n   - Gradually add water until a thick, smooth batter forms.\n   - Dip the cucumber and capsicum strips into the batter, ensuring they are evenly coated.\n\n3. Fry the vegetables:\n   - Heat oil in a deep pan or skillet.\n   - Fry the battered cucumber and capsicum strips until they turn golden brown and crispy.\n   - Remove from oil and drain on a paper towel to remove excess oil.\n\n4. Serve:\n   - Arrange the fried cucumber and capsicum strips on a serving platter.\n   - Sprinkle with a little sea salt for extra flavor.\n   - Serve with a side of soy sauce or your favorite dipping sauce.\n\nEnjoy this Asian-inspired vegan appetizer of crispy battered cucumber and capsicum that combines texture and flavor for a heavenly first course!"},
                {"role":"assistant","content":'''{"recipe_name": "Asian-inspired vegan appetizer", "ingredients": [{"name": "cucumber","amount": "1" ,"unit": "item"},{"name": "capsicum","amount": "1" ,"unit": "item"},{"name": "flour","amount": "1" ,"unit": "cup"},{"name": "soy sauce","amount": "1" ,"unit": "tablespoon"},{"name": "oil","amount": "1" ,"unit": "not specified"},{"name": "water","amount": "1" ,"unit": "not specified"},{"name": "sea salt","amount": "1" ,"unit": "not specified"}]}'''},
                {"role": "user", "content": recipe}
				]
)

In [42]:
recipe_extraction_json = json.loads(entity_extraction.choices[0].message.content)

In [43]:
recipe_extraction = pd.json_normalize(recipe_extraction_json, record_path =['ingredients'])

In [ ]:
#path_output = r"C:\Python\openai-lab\data\openfoodfacts.csv"
# path_output = r"C:\Python\data\openfoodfacts.csv" #Change path if required
path_output = r"C:\Python\openai-lab\support\notebooks\recipe_extraction.csv" #Change path if required

#df_products.to_excel(path_output, sheet_name='Sheet1', index = False)
recipe_extraction.to_csv(path_output, sep='\t', encoding='utf-8', index=False)